In [15]:
from vector import Vector
from decimal import Decimal, getcontext

getcontext().prec = 30

class Line(object):
    
    NO_NONZERO_ELTS_FOUND_MSG = "No nonzero elements found"
    
    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 2
        
        if not normal_vector:
            all_zeors = ['0'] * self.dimension
            normal_vector = Vector(all_zeors)
        self.normal_vector = normal_vector
        
        if not constant_term:
            constant_term = Decimal('0')
        self.constant_term = Decimal(constant_term)
        
        self.set_basepoint()

    def set_basepoint(self):
        try:
            n = self.normal_vector
            c = self.constant_term
            basepoint_coords = ['0'] * self.dimension
            
            initial_index = Line.first_nonzero_index(n)
            initial_coefficient = n[initial_index]
            
            basepoint_coords[initial_index] = c / initial_coefficient
            self.basepoint = Vector(basepoint_coords)
        except Exception as e:
            if str(e) == Line.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                raise e
                
    def __str__(self):
        num_decimal_places = 3
        
        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)
            output = ''
            
            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'
            if not is_initial_term:
                output += ' '
            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))
                
            return output
        
        n = self.normal_vector
        try:
            initial_index = Line.first_nonzero_index(n)
            terms = [
                write_coefficient(n[i], is_initial_term=(i==initial_index)) + 'x_{}'.format(i+1)
                for i in range(self.dimension) if round(n[i], num_decimal_places) != 0
                    ]
            output = ' '.join(terms)
            
        except Exception as e:
            if str(e) == Line.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e
                
        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)
        
        return output
    
    def is_parallel(self, l):
        return self.normal_vector.isParallel(l.normal_vector)
        
    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Line.NO_NONZERO_ELTS_FOUND_MSG)
        
class MyDecimal(Decimal):
    def is_near_zero(self, eps=10e-10):
        return abs(self) < eps


In [20]:
ln1 = Line(Vector([1,5.6]), 3)
print(ln1)
ln2 = Line(Vector([0,0]), 3)
print(ln1)
ln3 = Line(Vector([2,11.2]), 10)
ln1.is_parallel(ln3)

x_1 + 5.6x_2 = 3
x_1 + 5.6x_2 = 3


True